# Calculate biogeographic traits

Our focus is on the spatial distribution of microbial biogeographic traits, so we also need to calculate several microbial community measures. The [microgeo](https://github.com/ChaonanLi/microgeo) R package provides some basic functions for calculating microbial relative abundance, diversity, and community assembly mechanism indicators. Of course, you can also visualize more microbial traits onto maps, not limited to the traits calculated by the [microgeo](https://github.com/ChaonanLi/microgeo) R package.

Here we need three R packages for this section of [microgeo](https://github.com/ChaonanLi/microgeo) R package tutorial. Just run the following codes to import them into R environment.

In [1]:
suppressMessages(require("magrittr")) 
require("ggplot2")  %>% suppressMessages()
require("microgeo") %>% suppressMessages()

If the Chinese characters cannot be displayed correctly, please run the following codes to set locale to `UTF-8`:

In [2]:
prev_locale <- Sys.setlocale("LC_CTYPE", "C.UTF-8") 

We need a standard microgeo dataset for the presentations in the section of tutorial.

In [3]:
# Using the map downloaded from DataV.GeoAtlas
data(qtp)
map <- read_aliyun_map(adcode = c(540000, 630000, 510000)) %>% suppressMessages() 
dataset.dts.aliyun <- create_dataset(mat = qtp$asv, ant = qtp$tax, met = qtp$met, map = map,
                                     phy = qtp$tre, env = qtp$env, lon = "longitude", lat = "latitude") 
dataset.dts.aliyun %<>% rarefy_count_table() 
dataset.dts.aliyun %<>% tidy_dataset()
dataset.dts.aliyun %>% show_dataset()

ℹ [2024-01-08 21:19:00] INFO ==> all samples fall within the map area!

ℹ [2024-01-08 21:19:00] INFO ==> dataset has been created successfully!

ℹ [2024-01-08 21:19:00] INFO ==> use `object %>% show_dataset()` to check the summary of dataset.

ℹ [2024-01-08 21:19:04] INFO ==> the ASV/gene abundance table has been rarefied with a sub-sample depth of 5310



── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 1. Calculate the relative abundance

We implemented a function of `calc_rel_abund()`, which calculates the relative abundance of ASVs/genes at each of classification level. Just run the following codes.

In [4]:
# Calculate relative abundance
dataset.dts.aliyun %<>% calc_rel_abund() %>% suppressMessages()
head(dataset.dts.aliyun$abd$raw$Phylum[,1:5])

,p__,p__Acidobacteria,p__Actinobacteria,p__Armatimonadetes,p__Bacteroidetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,0.13182674,13.22034,48.90772,0.07532957,2.184557
s2,0.16949153,13.06968,51.35593,0.07532957,2.354049
s3,0.05649718,21.43126,43.76648,0.00000000,1.393597
s4,0.22598870,16.51601,47.25047,0.01883239,4.500942
s5,0.20715631,13.18267,55.93220,0.05649718,1.299435
s6,0.35781544,14.93409,46.34652,0.00000000,2.448211


In [5]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 2. Calculate the ecological markers

We also designed a function of `calc_markers()`, which identifies the potentially ecological markers base on the relative abundance of ASV/genes at a given classification level. If there is only one environmnetal variable for the identification, then a correlation method would be used. If the are two or more environmental variables, then a mantel test would be applied. **Please note that such a function only works when the relative abundance of ASVs/genes are available in microgeo dataset!**. 

In [6]:
# Identify ecological markers based on soil pH in <env>[Correlation]
dataset.dts.aliyun %<>% calc_markers(use.var = 'pH', annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') 

ℹ [2024-01-08 21:19:12] INFO ==> found 19 ecological markers at `Phylum` level..

✔ [2024-01-08 21:19:12] SAVE ==> results have been saved to: object$abd$mar



In [7]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Acidobacteria,p__Acidobacteria,-0.3899226,1.919089e-46
p__Verrucomicrobia,p__Verrucomicrobia,-0.3660577,9.731152e-41
p__Rokubacteria,p__Rokubacteria,-0.3382739,1.118185e-34
p__Actinobacteria,p__Actinobacteria,0.3194073,6.694358e-31
p__Chloroflexi,p__Chloroflexi,0.2969453,9.619864e-27
p__Gemmatimonadetes,p__Gemmatimonadetes,0.2933657,4.098446e-26


In [8]:
# Identify ecological markers based on soil pH and TOC in <env> [Mantel test]
dataset.dts.aliyun %<>% calc_markers(use.var = c('pH', 'TOC'), annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') 

ℹ [2024-01-08 21:19:12] INFO ==> mantel test would take a while...

ℹ [2024-01-08 21:19:12] INFO ==> current annot. name(1/26): p__

ℹ [2024-01-08 21:19:14] INFO ==> current annot. name(2/26): p__Acidobacteria

ℹ [2024-01-08 21:19:16] INFO ==> current annot. name(3/26): p__Actinobacteria

ℹ [2024-01-08 21:19:18] INFO ==> current annot. name(4/26): p__Armatimonadetes

ℹ [2024-01-08 21:19:20] INFO ==> current annot. name(5/26): p__Bacteroidetes

ℹ [2024-01-08 21:19:23] INFO ==> current annot. name(6/26): p__Chloroflexi

ℹ [2024-01-08 21:19:25] INFO ==> current annot. name(7/26): p__Crenarchaeota

ℹ [2024-01-08 21:19:27] INFO ==> current annot. name(8/26): p__Cyanobacteria

ℹ [2024-01-08 21:19:30] INFO ==> current annot. name(9/26): p__Deinococcus.Thermus

ℹ [2024-01-08 21:19:33] INFO ==> current annot. name(10/26): p__Elusimicrobia

ℹ [2024-01-08 21:19:36] INFO ==> current annot. name(11/26): p__Entotheonellaeota

ℹ [2024-01-08 21:19:38] INFO ==> current annot. name(12/26): p__Euryarchae

In [9]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Crenarchaeota,p__Crenarchaeota,0.2232591,0.001
p__Actinobacteria,p__Actinobacteria,0.1208577,0.001
p__Thaumarchaeota,p__Thaumarchaeota,0.1195895,0.001
p__Gemmatimonadetes,p__Gemmatimonadetes,0.1067845,0.001


In [10]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 3. Calculate alpha diversity indices

We implemented a functiobn of `calc_alpha_div()`, which calculates serveral alpha diversity indices such as observed ASVs and Shannon index. Just run the following codes.

In [11]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_alpha_div(measures = c("observed", "shannon")) %>% suppressMessages()

In [12]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 4. Calculate beta diversity distance matrices

We implemented a function of `calc_beta_div()`, which calculates serveral beta diversity distance matrix such as `Bray-Curtis` and `Jarccard` distance matrix. Just run the following codes.

In [13]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_beta_div(measures = c("bray", "jaccard")) %>% suppressMessages()

In [14]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


In [15]:
# Check results [Bray-Curtis distance]
dataset.dts.aliyun$div$bet$bray[1:4, 1:4]

,s1,s2,s3,s4
s1,0.0000000,0.4813559,0.5789077,0.4966102
s2,0.4813559,0.0000000,0.4813559,0.4107345
s3,0.5789077,0.4813559,0.0000000,0.4062147
s4,0.4966102,0.4107345,0.4062147,0.0000000


## 5. Calculate microbial community assembly indices

We implemeted a function of `calc_phylo_asmb()`, which calculates the metrics involved in microbial community assembly. Just run the following codes.

In [16]:
# Calculate `alpha.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'alpha.phylo', runs = 9, out.dir = 'test/calc_comm_asmb') %>% suppressMessages()

In [17]:
# Check results 
head(dataset.dts.aliyun$asb$alpha.phylo)

,ntaxa,mntd.obs,mntd.rand.mean,mntd.rand.sd,mntd.obs.rank,mntd.obs.z,mntd.obs.p,runs
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,994,0.04296439,0.08426412,0.007403890,1,-5.578113,0.1,9
s2,950,0.04789814,0.08316672,0.004549325,1,-7.752487,0.1,9
s3,845,0.04794208,0.09028277,0.004200045,1,-10.081008,0.1,9
s4,1028,0.04716382,0.08187408,0.004503587,1,-7.707249,0.1,9
s5,895,0.04621123,0.08886568,0.005389841,1,-7.913861,0.1,9
s6,983,0.04720200,0.08302965,0.005064797,1,-7.073857,0.1,9


In [18]:
# Calculate `beta.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'beta.phylo', runs = 9, out.dir = 'test/calc_comm_asmb') %>% suppressMessages()

In [19]:
# Check distance matrix 
names(dataset.dts.aliyun$asb$beta.phylo$dis) # distance matrices

[1] "b.mntd" "bc"     "rc"     "b.nti"

In [20]:
# Check the results 
head(dataset.dts.aliyun$asb$beta.phylo$raw$result) # the raw result of `iCAMP::qpen()`

,sample1,sample2,bMNTD,BC,bNTI,RC,process
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,s2,s1,0.01460073,0.4815443,-4.399015,-1.0000000,Homogeneous.Selection
2,s3,s1,0.01868585,0.5794727,-7.282901,-0.7777778,Homogeneous.Selection
3,s4,s1,0.01609248,0.4947269,-3.378989,-1.0000000,Homogeneous.Selection
4,s5,s1,0.01328793,0.3960452,-3.535605,-1.0000000,Homogeneous.Selection
5,s6,s1,0.01485771,0.4745763,-4.056332,-1.0000000,Homogeneous.Selection
6,s7,s1,0.01646125,0.5009416,-4.746696,-1.0000000,Homogeneous.Selection


In [21]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)

✔ object$asb$beta.phylo$raw$result: 7 beta phylogenetic assembling index/indices (sample1, sample2, bMNTD, ...)

✔ object$asb$beta.phylo$dis: 4 beta phylogenetic distance matrix/matrices (b.mntd, bc, rc, b.nti)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`
